此 ipynb 文件只用于内部演示，往 GitHub 提 PR 前会删除。

In [1]:
model_name = 'ced-mini'  # ['ced-tiny', 'ced-mini', 'ced-small', 'ced-base']

In [ ]:
# Convert checkpoint from original CED to 🤗 Transformers format
from transformers.models.ced.convert_ced_original_to_pytorch import convert_ced_checkpoint

model_saved_path = f'/home/zhangjunbo/sandbox/{model_name}'
convert_ced_checkpoint(model_name, model_saved_path)

In [3]:
import torch
import torchaudio

from transformers import CedConfig, CedForAudioClassification

In [ ]:
config = CedConfig(model_name)

DEVICE = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')

model = CedForAudioClassification(config).from_pretrained(model_saved_path).to(DEVICE).eval()

In [ ]:
wavpath = '/home/zhangjunbo/src/CED/resources/JeD5V5aaaoI_931_932.wav'
wave, _ = torchaudio.load(wavpath)
wave = wave.to(DEVICE)

output = model(wave).squeeze(0)
output, output.shape

In [6]:
import pandas as pd

label_maps = pd.read_csv('http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/class_labels_indices.csv').set_index('index')['display_name'].to_dict()

In [7]:
topk = 3

for k, (prob, label) in enumerate(zip(*output.topk(topk))):
    lab_idx = label.item()
    label_name = label_maps[lab_idx]
    print(f"Topk-{k+1} {label_name:<30} {prob:.4f}")

Topk-1 Finger snapping                0.8888
Topk-2 Slap, smack                    0.1020
Topk-3 Inside, small room             0.0365
